# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import scipy.integrate as integrate

import porepy as pp
import pygeon as pg

import sympy as sp

from math import ceil, floor, log10, exp

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


### Initial parameters definition

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.5e-2

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 100

quad_order = 1

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-10
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'compressed'

In [4]:
def initial_h(x): return 1

In [5]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1/n

In [6]:
# Time step
dt   = (T-0)/num_steps

### $\theta$ and $K$

In [7]:
# Symbolic psi
psi_var = sp.Symbol('psi', negative=True)

# Symbolic Theta
theta_expression = theta_r + (theta_s - theta_r) / (1 + (-alpha * psi_var) ** n) ** m
effective_saturation = (theta_expression - theta_r) / (theta_s - theta_r)

# Symbolic Conductivity K
hydraulic_conductivity_expression = K_s * (effective_saturation ** 0.5) * ( 1 - (1 - effective_saturation ** (1 / m)) ** m ) ** 2

In [8]:
# Theta lambda
theta_lambda = sp.lambdify(psi_var, theta_expression, 'numpy')

# Conductivity tensor lambda
conductivity_lambda = sp.lambdify(psi_var, hydraulic_conductivity_expression, 'numpy')

In [9]:
# Actual (and final) theta function
def theta(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * theta_s
    res[mask] = theta_lambda(psi[mask])

    return res

In [10]:
# Actual (and final) theta function
def conductivity(psi):
    if psi < 0:
        return conductivity_lambda(psi)
    else:
        return K_s

### Mesh and $V_h$

In [11]:
key = "flow"

In [12]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,1])
subdomain.compute_geometry()

# Convert it to a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])
mdg = pg.as_mdg(mdg)

In [13]:
h_field = pg.Lagrange1(key)

In [14]:
dof_h = darcy_dof = h_field.ndof(subdomain)

### BC

#### Dirichlet (essential)

In [15]:
right_bot = np.logical_and( subdomain.nodes[0, :] == 2, subdomain.nodes[1, :] < 1/3 )
left_top  = np.logical_and( subdomain.nodes[1, :] == 1, subdomain.nodes[0, :] < 1 )

dirichlet_flag  = np.logical_or(right_bot, left_top)
dirichlet_value = lambda t: np.array(right_bot, dtype=float) + np.array(left_top, dtype=float) * min( 3.2, 1 + 2.2 * t / dt_D )

#### Neumann (natural)

### Matrix Assembly

In [16]:
%%cython
import numpy as np

def find_ordering(coord: np.array):
    lx = np.argmin(coord[0, :])
    rx = np.argmax(coord[0, :])
    mx = np.setdiff1d(np.array([0,1,2]), np.array([lx, rx]))[0]

    # Vertical Alignment
    if np.abs( coord[0, lx] - coord[0, mx] ) < 1e-7:
        # lx and mx vertical aligned, rx no
        up =   lx if np.argmax(coord[1, np.array([lx, mx])]) == 0 else mx
        down = lx if np.argmin(coord[1, np.array([lx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, rx] ) < 1e-7:
            return [up, down, rx]
        else:
            return [down, rx, up]
    else:
        # rx and mx vertical aligned, lx no
        up =   rx if np.argmax(coord[1, np.array([rx, mx])]) == 0 else mx
        down = rx if np.argmin(coord[1, np.array([rx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, lx] ) < 1e-7:
            return [up, lx, down]
        else:
            return [down, up, lx]

In [17]:
%%cython
import numpy as np

def K_func_eval(base_height: float, base_width: float, element_height: float, element_width: float, m: int, ls_eta: float, rs_eta: float, grad_eta: float, x, y):
    coord = lambda t: ((m+1) * (1-t) - (m-1) * t) / 2

    x3 = base_height + coord(x) * element_height
    eta = (1-coord(y)) * ls_eta + coord(y) * rs_eta

    chi_x3  = eta
    chi_eta = x3

    return np.array([[             chi_x3,                            -chi_eta * grad_eta],
                     [-chi_eta * grad_eta, (1 + np.power(chi_eta * grad_eta, 2)) / chi_x3]])

    # This is the full version:
    #return np.array([[                  chi_x3 * k11,                                                   k12 - chi_eta * grad_eta * k11],
    #                 [k21 - chi_eta * grad_eta * k11, ( k22 + chi_eta * grad_eta * ( chi_eta * grad_eta * k11 - k12 - k21 ) ) / chi_x3]])

In [18]:
def local_A(coord, psi, m):

    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]
    
    J_T_1_T = np.array([[x2[1]-x0[1], x0[1]-x1[1]],
                        [x0[0]-x2[0], x1[0]-x0[0]]]) / ((x1[0]-x0[0]) * (x2[1]-x0[1]) - (x2[0]-x0[0]) * (x1[1]-x0[1]))
    

    q_funcs = [J_T_1_T @ np.array([-1, -1]), J_T_1_T @ np.array([ 1, 0]), J_T_1_T @ np.array([0,  1])]

    M = np.zeros(shape=(3,3))

    jacobian = 1 / np.linalg.det( J_T_1_T.T )

    ordered_psi = psi[ordering]

    psi_fun = lambda x,y: ordered_psi[0] + (ordered_psi[1] - ordered_psi[0]) * x + (ordered_psi[2] - ordered_psi[0]) * y
    #K_func = lambda x,y: K_func_eval(min(coord[1, :]), min(coord[0, :], element_height, element_width), m, 1, 1, 0, x, y)

    for i in range(3):
        for j in range(3):
            integrand = lambda ys,x: np.array([conductivity(psi_fun(x,y)) for y in np.array(ys)]) # 
            inside = lambda xs, n: np.array([integrate.fixed_quad(integrand, 0, 1-x, args=(x,), n=n)[0] for x in np.array(xs)])
            tmp = integrate.fixed_quad(inside, 0, 1, n=quad_order, args=(quad_order,))[0]

            M[ ordering[i], ordering[j] ] = q_funcs[j].T @ np.array([[1, 0], [0, 1/9]]) @ q_funcs[i] * jacobian * tmp

    return M

In [19]:
local_A(np.array([[0, 1, 0], [0, 0, 1]]), np.array([1,1,1]), np.prod(np.array([-1, 1, -1])))

array([[ 0.02755556, -0.0248    , -0.00275556],
       [-0.0248    ,  0.0248    ,  0.        ],
       [-0.00275556,  0.        ,  0.00275556]])

In [20]:
subdomain, data = mdg.subdomains(return_data=True)[0]

pp.initialize_data(subdomain, data, key, {"second_order_tensor": pp.SecondOrderTensor(np.ones(subdomain.num_cells))})

{'discretization_matrices': {'flow': {}},
 'parameters': Data object for physical processes flow
 The keyword "flow" has the following parameters specified: second_order_tensor}

In [21]:
def stifness(psi):

    # Map the domain to a reference geometry (i.e. equivalent to compute
    # surface coordinates in 1d and 2d)

    _, _, sign = sps.find(subdomain.cell_faces)
    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.
    size = np.power(subdomain.dim + 1, 2) * subdomain.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    cell_nodes = subdomain.cell_nodes()

    for c in np.arange(subdomain.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]


        # Compute the stiff-H1 local matrix
        A = local_A(coord_loc, psi[nodes_loc], np.prod(sign[loc]))

        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))
        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

In [22]:
M_h = h_field.assemble_mass_matrix(subdomain)

In [23]:
proj_psi = h_field.eval_at_cell_centers(subdomain)

In [24]:
M_h.todense()

matrix([[0.00055556, 0.00013889, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00013889, 0.00083333, 0.00013889, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.00013889, 0.00083333, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.00083333, 0.00013889,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.00013889, 0.00083333,
         0.00013889],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00013889,
         0.00055556]])

### Solve system

In [25]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [26]:
def save_step(sol, saver, i):
    ins = list()

    ins.append((subdomain, "cell_h", proj_psi @ sol))
    ins.append((subdomain, "cell_p", proj_psi @ (sol - subdomain.nodes[1,:] * 3)))

    saver.write_vtu(ins, time_step=i)

In [27]:
sol = h_field.interpolate(subdomain, initial_h)

In [28]:
# Save the initial solution
saver = pp.Exporter(mdg, 'sol', folder_name=output_directory)
save_step(sol, saver, 0)

In [29]:
csv_base = os.path.join(output_directory, 'csv')

if os.path.exists(csv_base):
    shutil.rmtree(csv_base)
    
os.mkdir(csv_base)

In [30]:
for time_instant_index in range(1, num_steps+1):

    time = time_instant_index * dt
    print('Time ' + str(round(time, 5)))

    prev = sol

    time_rhs = np.zeros(dof_h)
    time_rhs = M_h @ theta(prev - subdomain.nodes[1,:] * 3)

    for k in range(K):
        spp = L * M_h + dt * stifness(prev - subdomain.nodes[1,:] * 3)
        rhs = time_rhs.copy()

        rhs += L * M_h @ prev - M_h @ theta(prev - subdomain.nodes[1,:] * 3)

        ls = pg.LinearSystem(spp, rhs)
        ls.flag_ess_bc( dirichlet_flag, dirichlet_value(time) )

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ M_h @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ M_h @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break
        
    print('')    
    sol = current
    
    save_step(sol, saver, time_instant_index)

    export_name = os.path.join(csv_base, str(time_instant_index) + '.csv')

    with open( export_name, 'w' ) as file:

        file.write('x,y,h,p\n')

        for x,y,h,p in zip( subdomain.nodes[0, :], subdomain.nodes[1, :], sol, sol - subdomain.nodes[1, :]):
            file.write(f'{x},{y},{h},{p}\n')



saver.write_pvd([n * dt for n in range(num_steps + 1)])

Time 0.02083
Iteration #001, error L2 relative psi:    0.08107727710292


Iteration #002, error L2 relative psi:    0.00811466089050
Iteration #003, error L2 relative psi:    0.00026542428475
Iteration #004, error L2 relative psi:    0.00004169870057
Iteration #005, error L2 relative psi:    0.00000814364024

Time 0.04167
Iteration #001, error L2 relative psi:    0.10987045516623
Iteration #002, error L2 relative psi:    0.02763471255395
Iteration #003, error L2 relative psi:    0.00335195964156
Iteration #004, error L2 relative psi:    0.00011671137421
Iteration #005, error L2 relative psi:    0.00002607222973
Iteration #006, error L2 relative psi:    0.00000292170892

Time 0.0625
Iteration #001, error L2 relative psi:    0.15372437985904
Iteration #002, error L2 relative psi:    0.05316983931153
Iteration #003, error L2 relative psi:    0.01475064986440
Iteration #004, error L2 relative psi:    0.00203152506937
Iteration #005, error L2 relative psi:    0.00073755393376
Iteration #006, error L2 relative psi:    0.00019631470610
Iteration #007, error L2 rela

KeyboardInterrupt: 

for time_instant_index in range(1, num_steps+1):

    time = time_instant_index * dt
    print('Time ' + str(round(time, 5)))

    prev = sol

    time_rhs = np.zeros(dof_h)
    #time_rhs = M_h @ theta(prev - subdomain.nodes[1,:] * 3)

    spp = L / dt * M_h + stifness(prev - subdomain.nodes[1,:] * 3)
    rhs = time_rhs.copy()

    rhs += L / dt * M_h @ prev #- M_h @ theta(prev - subdomain.nodes[1,:] * 3)
    rhs += neumann_value(time)

    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc( dirichlet_flag, dirichlet_value(time) )

    current = ls.solve()

    print('')    
    sol = current
    
    save_step(sol, saver, time_instant_index)

    export_name = os.path.join(csv_base, str(time_instant_index) + '.csv')

    with open( export_name, 'w' ) as file:

        file.write('x,y,h,p\n')

        for x,y,h,p in zip( subdomain.nodes[0, :], subdomain.nodes[1, :], sol, sol - subdomain.nodes[1, :]):
            file.write(f'{x},{y},{h},{p}\n')


saver.write_pvd([n * dt for n in range(num_steps + 1)])